<a href="https://colab.research.google.com/github/Nihalakshay2004/Fighting-Fabrication-/blob/main/Fighting%20_fabrication_using_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
manjilkarki_deepfake_and_real_images_path = kagglehub.dataset_download('manjilkarki/deepfake-and-real-images')

print('Data source import complete.')


In [ ]:
manjilkarki_deepfake_and_real_images_path

'/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1'

In [ ]:
ls /root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1/Dataset


Test/  Train/  Validation/


In [ ]:
import tensorflow as tf
from keras import layers
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import numpy as np
import cv2
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.callbacks import Callback , ReduceLROnPlateau , ModelCheckpoint, CSVLogger
from sklearn.metrics import cohen_kappa_score, accuracy_score
from tensorflow.keras.losses import categorical_crossentropy as logloss
from tensorflow.keras.metrics import categorical_accuracy
import pandas as pd
import os
from PIL import Image
import scipy
from sklearn import metrics
from sklearn.model_selection import train_test_split
import json

In [ ]:
train_dir = '/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1/Dataset/Train'
valid_dir = '/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1/Dataset/Validation'
test_dir = '/root/.cache/kagglehub/datasets/manjilkarki/deepfake-and-real-images/versions/1/Dataset/Test'

In [ ]:
# View an image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os

def view_random_image(target_dir, target_class):
  target_folder = target_dir + "/" + target_class
  print(f"target folder {target_folder}")
  random_image = random.sample(os.listdir(target_folder), 1)
  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off");

  print(f"Image shape: {img.shape}") # show the shape of the image

  return img

# View random image
# img = view_random_image(target_dir=train_dir,
#                         target_class="Real")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)

train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(valid_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

test_data = test_datagen.flow_from_directory(test_dir,
                                               batch_size=1,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               shuffle=False,
                                               seed=42)

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.


# model1- VGG

In [ ]:
# End to End TinyVGG Model

tf.random.set_seed(42)


#Create a CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=20,
                         kernel_size=3, # can also be (3, 3)
                         activation="relu",
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(20, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(20, 3, activation="relu"),
  tf.keras.layers.Conv2D(20, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(4),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=5, #5, 8
                        steps_per_epoch=128, # 44, 99, 128, 4k XD
                        validation_data=valid_data,
                        validation_steps=128)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
128/128 ━━━━━━━━━━━━━━━━━━━━ 354s 3s/step - accuracy: 0.5186 - loss: 0.6921 - val_accuracy: 0.5273 - val_loss: 0.7186
Epoch 2/5
105/128 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.5845 - loss: 0.6710

# Save the model
model_1.save("VGG_1.h5")

In [ ]:
def load_and_prep_image(filename, img_shape=224):
  # Read in target file (an image)
  print(filename)
  img = tf.io.read_file(filename)

  img = tf.image.decode_image(img, channels=3)
  img = tf.image.resize(img, size = [img_shape, img_shape])
  img = img/255.
  return img

# print(test_data[0])

# # print(test_data)


# pred = model_1.predict(test_data[9000])
# ctr = 1 if pred > .50 else 0
# print()
# pred

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix




# func to generate different score and confusion matrix from test score.
def gen_test_data(model):
    # Generate Confusion Matrix
    y_pred = model.predict(test_data)
    y_test = test_data.classes

#     y_pred_labels = np.argmax(y_pred, axis=1)
    y_pred_labels = (y_pred > 0.5).astype(np.float32)

    confusion_mat = confusion_matrix(y_test, y_pred_labels)
    print(confusion_mat)

    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_mat, display_labels = [False, True])

    cm_display.plot()
    plt.show()

    # Get AUC, ROC and Precision Score
    print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))
    print("AP Score:", metrics.average_precision_score(y_test, y_pred))
    print()
    print(metrics.classification_report(y_test, y_pred_labels))
    _, accu = model.evaluate(test_data)
    print('Final Test Acccuracy = {:.3f}'.format(accu*100))

gen_test_data(model_1)

In [ ]:
#Save the model
# model_1.save("VGG_1.h5")

# def load_and_prep_image(filename, img_shape=224):
#   # Read in target file (an image)
#   print(filename)
#   img = tf.io.read_file(filename)

#   img = tf.image.decode_image(img, channels=3)
#   img = tf.image.resize(img, size = [img_shape, img_shape])
#   img = img/255.
#   return img

# test_img = load_and_prep_image(f"{test_dir}/Real/real_4.jpg")
# test_img = tf.expand_dims(test_img, axis=0)

import keras
model_1.save("/kaggle/working/VGG_01_FINAL.keras")

# reconstructed_model = keras.models.load_model("/kaggle/working/vgg_01.keras")

# # Let's check:
# print(np.testing.assert_allclose(
#     model_1.predict(test_img), reconstructed_model.predict(test_img)
# ))

In [ ]:
# from keras.models import load_model
# vgg_1 = load_model('/kaggle/working/VGG_1.h5')


In [ ]:
# def load_and_prep_image(filename, img_shape=224):
#   # Read in target file (an image)
#   print(filename)
#   img = tf.io.read_file(filename)

#   img = tf.image.decode_image(img, channels=3)
#   img = tf.image.resize(img, size = [img_shape, img_shape])
#   img = img/255.
#   return img

# test_img = load_and_prep_image(f"{test_dir}/Real/real_4.jpg")
# test_img = tf.expand_dims(test_img, axis=0)
# vgg_1.predict(test_img)

In [ ]:
# import pandas as pd
# pd.DataFrame(history_1.history).plot(figsize=(10, 7));

In [ ]:
# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
plot_loss_curves(history_1)

# CNN

In [ ]:
# model_2 with a MaxPool2D layer after every Conv2D layer

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Activation
from tensorflow.keras import Sequential

tf.random.set_seed(42)

model_2 = Sequential([
  Conv2D(20, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2), # reduce number of features by half
  Conv2D(20, 3, activation='relu'),
  MaxPool2D(),
  Conv2D(20, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(1, activation='sigmoid')
])

model_2.compile(loss='binary_crossentropy',
                optimizer=Adam(),
                metrics=['accuracy'])

history_2 = model_2.fit(train_data,
                        epochs=6,
                        steps_per_epoch=256,
                        validation_data=valid_data,
                        validation_steps=256)

In [ ]:
plot_loss_curves(history_2)

In [ ]:
model_2.save("VGG_2_FINAL.h5")

In [ ]:
# TODO: Try DataAugmentation during image preprocessing.
gen_test_data(model_2)

In [ ]:
# def load_and_prep_image(filename, img_shape=224):
#   # Read in target file (an image)
#   print(filename)
#   img = tf.io.read_file(filename)

#   img = tf.image.decode_image(img, channels=3)
#   img = tf.image.resize(img, size = [img_shape, img_shape])
#   img = img/255.
#   return img

In [ ]:
# train_img_path_1 = f"{test_dir}/Real/real_4.jpg"
# train_img_path_2 = f"{test_dir}/Fake/fake_12.jpg"
# train_img_real_1 = load_and_prep_image(train_img_path_1)
# train_img_real_2 = load_and_prep_image(train_img_path_2)
# train_img_real_1 = tf.expand_dims(train_img_real_1, axis=0)
# train_img_real_2 = tf.expand_dims(train_img_real_2, axis=0)

In [ ]:
# pred_1 = model_2.predict(train_img_real_1)
# pred_2 = model_2.predict(train_img_real_2)
# print(pred_1)
# print(pred_2)

# Resnet 50

In [ ]:
input_shape=(224,224,3)
batch_size=64


def build_model():
    res_densenet = ResNet50(
                        weights='imagenet',
                        include_top=False,
                        input_shape=input_shape
                        )
    res_model = Sequential([res_densenet,
                        layers.GlobalAveragePooling2D(),
                        layers.Dense(64,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dropout(0.5),
                        layers.Dense(1, activation='sigmoid')
                        ])
    res_model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy']
                 )
    return res_model


model_resnet = build_model()
model_resnet.summary()

In [ ]:
tf.random.set_seed(42)
history_3 = model_resnet.fit(train_data,
    epochs = 8,
    steps_per_epoch = 256,
    validation_data =valid_data,
    validation_steps = 256
)

In [ ]:
plot_loss_curves(history_3)

In [ ]:
# ResNet50_1 Save
import keras
model_resnet.save("/kaggle/working/Res_01_FINAL.keras")

# reconstructed_model = keras.models.load_model("/kaggle/working/Res_01.keras")

# # Let's check:
# np.testing.assert_allclose(
#     model.predict(test_input), reconstructed_model.predict(test_input)
# )

# filename = 'ResNet50_1.pkl'
# pickle.dump(model_resnet, open(filename, 'wb'))

In [ ]:
# def load_and_prep_image(filename, img_shape=224):
#   # Read in target file (an image)
#   print(filename)
#   img = tf.io.read_file(filename)

#   img = tf.image.decode_image(img, channels=3)
#   img = tf.image.resize(img, size = [img_shape, img_shape])
#   img = img/255.
#   return img

# test_img = load_and_prep_image(f"{test_dir}/Fake/fake_10.jpg")
# test_img = tf.expand_dims(test_img, axis=0)
# # vggyo.predict(test_img)
# model_resnet.predict(test_img)

In [ ]:
# from keras.models import load_model
# vggyo = load_model('/kaggle/working/Res_0100.keras')

# def load_and_prep_image(filename, img_shape=224):
#   # Read in target file (an image)
#   print(filename)
#   img = tf.io.read_file(filename)

#   img = tf.image.decode_image(img, channels=3)
#   img = tf.image.resize(img, size = [img_shape, img_shape])
#   img = img/255.
#   return img

# test_img = load_and_prep_image(f"{test_dir}/Real/real_4.jpg")
# test_img = tf.expand_dims(test_img, axis=0)
# vggyo.predict(test_img)

gen_test_data(model_resnet)

In [ ]:
def build_model_2():
    densenet = ResNet50(
                        weights='imagenet',
                        include_top=False,
                        input_shape=input_shape
                        )
    model = Sequential([densenet,
                        layers.GlobalAveragePooling2D(),
                        layers.Dense(512,activation='relu'),
                        layers.Dense(128,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dense(64,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dense(64,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dropout(0.5),
                        layers.Dense(128,activation='relu'),
                        layers.Flatten(),
                        layers.Dense(128,activation='relu'),
                        layers.Dense(1, activation='sigmoid')
                        ])
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy']
                 )
    return model


model_resnet_2 = build_model_2()
model_resnet_2.summary()

In [ ]:
tf.random.set_seed(42)
history_4 = model_resnet_2.fit(train_data,
    epochs = 16,
    steps_per_epoch = 128,
    validation_data =valid_data,
    validation_steps = 126
)

In [ ]:
plot_loss_curves(history_4)

In [ ]:
gen_test_data(model_resnet_2)

In [ ]:
# ResNet50_1 Save
import keras
model_resnet_2.save("/kaggle/working/Res_02_FINAL.keras")

In [ ]:
# ResNet50_2 Save

# model_resnet_2.save("ResNet50_2.h5")

# tf.random.set_seed(42)
# history_resnet_2 = model_resnet_2.fit(train_data,
#     epochs = 6,
#     steps_per_epoch = 1024,
#     validation_data =valid_data,
#     validation_steps = 1024
# )


In [ ]:
# plot_loss_curves(history_resnet_2)

# InceptionNet

In [ ]:
from keras.applications.inception_v3 import InceptionV3

input_shape=(224,224,3)
batch_size=64

def build_model_inception():
    densenet = InceptionV3(
                        weights='imagenet',
                        include_top=False,
                        input_shape=input_shape,
                        )
    model = Sequential([densenet,
                        layers.GlobalAveragePooling2D(),
                        layers.Dense(512,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dense(256,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dense(128,activation='relu'),
                        layers.BatchNormalization(),
                        layers.Dropout(0.5),
                        layers.Dense(1, activation='sigmoid')
                        ])
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy']
                 )
    return model

model_icv3 = build_model_inception()
model_icv3.summary()

In [ ]:
tf.random.set_seed(42)
history_5 = model_icv3.fit(train_data,
    epochs = 8,
    steps_per_epoch = 1024,
    validation_data =valid_data,
    validation_steps = 1024
)

In [ ]:
plot_loss_curves(history_5)

In [ ]:
# InceptionNetV3_1 Save
# ResNet50_1 Save
import keras
model_icv3.save("/kaggle/working/ICV3_FINAL.keras")


# filename = 'InceptionNetV3_1.pkl'
# pickle.dump(model_icv3, open(filename, 'wb'))

In [ ]:
# tf.random.set_seed(42)
# history_icv3_2 = model_icv3.fit(train_data,
#     epochs = 24,
#     steps_per_epoch = 1024,
#     validation_data =valid_data,
#     validation_steps = 1024
# )

gen_test_data(model_icv3)

In [ ]:
# plot_loss_curves(history_icv3_2)

In [ ]:
model_1.summary()

In [ ]:
# Initialize a dictionary to store losses
model_losses = {}

# Fit models and store the losses

model_losses['model1'] = history_1.history['loss']


model_losses['model2'] = history_2.history['loss']


model_losses['model3'] = history_3.history['loss']

model_losses['model4'] = history_4.history['loss']

model_losses['model5'] = history_5.history['loss']


In [ ]:
model_losses